<a href="https://colab.research.google.com/github/brandonko/FairnessNLP/blob/main/Bias_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bias Evaluation Metrics**

### Co-occurrence Bias in the Dataset
Metric defined in [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

### Embedding Bias
Metric defined in [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

### Counterfactual Evaluation of Sentiment Bias
Metric defined in [Huang et al. (2020)](https://aclweb.org/anthology/2020.findings-emnlp.7.pdf)

### (epsilon-k)-Fairness
Metric defined in [Ma et al. (2020)](https://www.semanticscholar.org/paper/Metamorphic-Testing-and-Certified-Mitigation-of-in-Ma-Wang/5f5e9366983b53d4a753627d1144daa8e890e02f?p2df)